## 목표

- Kobert-Transformers 대신 KoELECTRA를 사용해 CNN 모델과 각각 특징을 추출하여 소프트보팅 앙상블 모델을 만든다
- KFold를 이용해 다양한 학습을 하게 하여 성능을 높인다

### 순서
- 1. 필요 모듈들 설치 및 임포트
- 2. 기본 데이터 불러오기
- 3. 필요한 함수들 설정(모델 포함)
- 4. 모델 매개변수 설정
- 5. 모델학습
- 6. 평가
- 7. 예측

#### 1. 필요 모듈들 설치 및 임포트

In [2]:
# !pip install scikeras --users
# !pip install nlpaug

In [17]:
import librosa
# import librosa.display
# import IPython.display as ipd
# from IPython.display import Audio

# from random import randint
import numpy as np
import pandas as pd
# import pydub #오디오 파일을 다루는 라이브러리 : 변환 조작 재생 분석 등
# import gTTS # 텍스트를 음성으로 변환하는 라이브러리. 나는 당장 쓸 일 없을 듯?

from tqdm import tqdm, tqdm_notebook
# import np_utils
# import matplotlib.pyplot as plt
# import seaborn as sns

# import nlpaug.augmenter.audio as naa
# import wave

import glob
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ['TORCH_USE_CUDA_DSA'] = '1'

import winsound as ws
import json
# import shutil
# import sys
# import logging
# import unicodedata
# from shutil import copyfile
# import warnings
# warnings.filterwarnings("always")


from sklearn.preprocessing import LabelEncoder,StandardScaler, MinMaxScaler, RobustScaler, FunctionTransformer
# from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import recall_score,precision_score
from sklearn.model_selection import train_test_split, KFold
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from scikeras.wrappers import KerasClassifier 

from sklearn.metrics import accuracy_score, f1_score
# from sklearn.svm import SVC 
# from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline

import joblib
from joblib import dump, load
# from xgboost import XGBClassifier 

# import keras
import tensorflow as tf
from keras.callbacks import ReduceLROnPlateau,ModelCheckpoint,EarlyStopping, Callback
from keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Dense, Conv1D, MaxPooling1D, Flatten, Dropout, concatenate
# from tensorflow.keras.optimizers import Adam
# from keras.preprocessing.sequence import pad_sequences
# from keras.utils import to_categorical
# from sentence_transformers import SentenceTransformer
from keras.metrics import Precision, Recall, F1Score

import torch
from torch import nn
import torch.optim as optim
from torch.optim import AdamW
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# np.bool = np.bool_

# import mxnet
# import gluonnlp as nlp

# from transformers import AdamW
# from transformers.optimization import get_cosine_schedule_with_warmup
# from transformers import BertModel

from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift, Shift # 음성 데이터 증강용



#### 2. 기본 데이터 불러오기

In [18]:
train_csv = pd.read_csv('e:/Data2/csv/train.csv')
test_csv = pd.read_csv('e:/Data2/csv/test.csv')
exam_list = ['5fbe364744697678c497c07f','5fbe361844697678c497c07e']

train_audio_path = 'e:/Data2/train/'
test_audio_path = 'e:/Data2/test/'
exam_audio_path = 'e:/Data2/exam/'

# X = np.load('./features/train_features_6.npy')
X = np.load('./features/train_features_8.npy')  #6번 특징이 별로 안 좋을 경우 시도
X = np.real(X) #복소수 형태로 값이 나왔음
y = train_csv.iloc[:,-7:]
y= np.array(y)

# X_val = np.load('./features/test_features_6.npy')
X_test = np.load('./features/test_features_8.npy')
X_test = np.real(X_test)
y_test = test_csv.iloc[:,-7:]
y_test = np.array(y_val)

#### 3. 필요한 함수들 설정(모델 포함)
- 비프음 함수
- 음성 특징 학습 함수(CNN)
- 문자 특징 학습 모델(KoELECTRA)

In [19]:
#에러 발생 혹은 작업 완료시 beep로 알리기
def beepError():
    fr = 1000    # range : 37 ~ 32767
    du = 2000     # 1000 ms ==1second
    ws.Beep(fr, du) # winsound.Beep(frequency, duration)

def beepSuccess():
    fr = 2000    # range : 37 ~ 32767
    du = 1000     # 1000 ms ==1second
    ws.Beep(fr, du) # winsound.Beep(frequency, duration)

In [20]:
# f1 스코어를 쓸 수 있게 만드는 함수
# f1 스코어의 결과 타입은 tensor이므로 이걸 변환해 주는 작업 필요.

# class F1ScoreCallback(Callback):
#     def __init__(self, validation_data, save_best_only=True):
#         super(F1ScoreCallback, self).__init__()
#         self.X_val, self.y_val = validation_data
#         self.save_best_only = save_best_only
#         self.best_f1 = 0

#     def on_epoch_end(self, epoch, logs=None):
#         y_pred = np.argmax(self.model.predict(self.X_val), axis=1)
#         f1_val = f1_score(np.argmax(self.y_val, axis=1), y_pred, average='weighted')
#         print(f' - val_f1_score: {f1_val:.4f}')
#         if f1_val > self.best_f1:
#             self.best_f1 = f1_val
#             self.model.save('./model/best_model.keras')
#             print("Best F1 score. Model saved.")
#         elif not self.save_best_only:
#             self.model.save('./model/best_model.keras')
#             print("Model saved.")
#         if logs is not None:
#             logs['val_f1_score'] = f1_val

In [21]:
# 음성 모델
def cnn_model(data):
    model=Sequential()
    model.add(InputLayer(shape=(data.shape[1],data.shape[2])))
    model.add(Conv1D(256, kernel_size=5, strides=1, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

    model.add(Conv1D(256, kernel_size=5, strides=1, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

    model.add(Conv1D(128, kernel_size=5, strides=1, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))
    model.add(Dropout(0.2))

    model.add(Conv1D(64, kernel_size=5, strides=1, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

    model.add(Flatten())
    model.add(Dense(units=32, activation='relu'))
    model.add(Dropout(0.3))

    model.add(Dense(units=7, activation='softmax'))
    model.compile(optimizer = 'adamW' , loss = 'categorical_crossentropy' , metrics = ['accuracy','f1_score']) #metrics 추가

    return model

In [22]:
class ElectraForMultiClassClassification(nn.Module):
    def __init__(self, num_classes):
        super(ElectraForMultiClassClassification, self).__init__()
        self.electra = ElectraModel.from_pretrained("monologg/koelectra-base-v3-discriminator")
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(self.electra.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.electra(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0]  # [CLS] 토큰의 표현
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits

In [23]:
class Text_Input(Dataset):

    def __init__(self, data):
        self.data = data
        self.data.loc[(train_csv['감정'] == "Fear"), '감정'] = 0
        self.data.loc[(train_csv['감정'] == "Surprise"), '감정'] = 1
        self.data.loc[(train_csv['감정'] == "Angry"), '감정'] = 2
        self.data.loc[(train_csv['감정'] == "Sadness"), '감정'] = 3
        self.data.loc[(train_csv['감정'] == "Neutral"), '감정'] = 4
        self.data.loc[(train_csv['감정'] == "Happiness"), '감정'] = 5
        self.data.loc[(train_csv['감정'] == "Disgust"), '감정'] = 6
        self.tokenizer =  ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")


    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx, 1:3].values
        text = row[0]
        y = row[1]

        inputs = self.tokenizer(
            text,
            return_tensors='pt',
            truncation=True,
            max_length=256,
            padding='max_length',
            add_special_tokens=True
            )

        input_ids = inputs['input_ids'][0]
        attention_mask = inputs['attention_mask'][0]

        return input_ids, attention_mask, y

In [24]:
# 기본적인 사전 설정
rlrp = ReduceLROnPlateau(monitor='loss', factor=0.4, verbose=0, patience=5, min_lr=0.0000001) #learning rate 조절 
modelpath = './model/model_{epoch:02d}-{val_accuracy:.4f}.keras'
# modelpath = './model/model_{epoch:02d}-{f1_score:.4f}.keras'


mcp = ModelCheckpoint(
    modelpath,     #저장할 모델의 경로
  monitor = 'val_accuracy', #val_f1_score를 기준으로 전보다 모델이 나아지는 걸 확인
 
  save_best_only = True,    #나아진 결과만 저장
#     save_weights_only=True , #이걸 써 줘야 weights.h5로 저장 가능하다.
  verbose = 1               #과정을 출력
)

#전보다 나아지지 않으면 학습중단

es = EarlyStopping(
    monitor = 'val_accuracy',
#     mode='max',
    patience = 5,    # 전보다 나아지지 않아도 실행할 횟수
    restore_best_weights=True
)


In [28]:
joblib.dump(scaler,'./scaler/rscaler.pkl')

['./scaler/rscaler.pkl']

In [27]:
kf = KFold(n_splits=5, shuffle=True)
best_score = 0
best_model = None

# KFold를 통한 데이터 분할 및 스케일링
for train_index, test_index in kf.split(X):
    X_train, X_valid = X[train_index], X[test_index]
    y_train, y_valid = y[train_index], y[test_index]
    
    # 스케일링을 위한 StandardScaler 객체 생성 및 학습 데이터에 fit
#     scaler = StandardScaler()
#     scaler = MinMaxScaler()
    scaler = RobustScaler()
    scaler = scaler.fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    
    # 학습 데이터와 동일한 스케일링을 테스트 데이터에 적용
    X_valid_scaled = scaler.transform(X_valid)
    
    X_train_scaled  = np.expand_dims(X_train_scaled,axis =1)
    X_valid_scaled = np.expand_dims(X_valid_scaled ,axis =1)
    
    # 모델 학습
    model = cnn_model(X_train_scaled)
    history=model.fit(
    X_train_scaled,
    y_train,batch_size=32, #32일 때 약 35% 성능, 16일 때 약 35.2% 성능, 8일 때는 약 41.4%
    epochs=50, 
    validation_data=(X_valid_scaled , y_valid), 
    callbacks=[rlrp,mcp,es]
    )
    
    # 모델 평가
    val_loss, val_acc, val_f1_score = model.evaluate(X_valid_scaled, y_valid)
    
    X_test_scaled = scaler.transform(X_test)
    X_test_scaled = np.expand_dims(X_test_scaled  ,axis =1)
    
    test_loss, test_acc, test_f1_score = model.evaluate(X_test_scaled, y_test)
    
    print("Valid Loss: ",val_loss, end=', ')
    print("Valid f1_score: ",val_f1_score, end=', ')
    print("Valid Accuracy: ",val_acc, end=', ')
    print("Test Loss: ",test_loss, end=', ')
    print("Test f1_score: ",test_f1_score, end=', ')
    print("Test Accuracy: ",test_acc, end=', ')
    a
    
    if test_acc>best_score:
        best_score = test_acc
        best_model = model
        print('더 나은 test_acc 모델 저장!')
        
beepSuccess()

Epoch 1/50
990/990 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.3732 - f1_score: 0.1212 - loss: 1.6757
Epoch 1: val_accuracy improved from -inf to 0.43256, saving model to ./model/model_01-0.4326.keras
990/990 ━━━━━━━━━━━━━━━━━━━━ 17s 15ms/step - accuracy: 0.3732 - f1_score: 0.1213 - loss: 1.6757 - val_accuracy: 0.4326 - val_f1_score: 0.2029 - val_loss: 1.5448 - learning_rate: 0.0010
Epoch 2/50
990/990 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.4200 - f1_score: 0.1948 - loss: 1.5601
Epoch 2: val_accuracy improved from 0.43256 to 0.44254, saving model to ./model/model_02-0.4425.keras
990/990 ━━━━━━━━━━━━━━━━━━━━ 15s 16ms/step - accuracy: 0.4200 - f1_score: 0.1948 - loss: 1.5601 - val_accuracy: 0.4425 - val_f1_score: 0.2139 - val_loss: 1.5122 - learning_rate: 0.0010
Epoch 3/50
990/990 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.4448 - f1_score: 0.2154 - loss: 1.5113
Epoch 3: val_accuracy improved from 0.44254 to 0.44923, saving model to ./model/model_03-0.4492.keras
990/990 ━

NameError: name 'val_loss' is not defined

In [ ]:
best_score

In [ ]:
train_text, valid_text = train_test_split(train_csv, test_size=0.3, random_state=42, shuffle=True, stratify=train_csv['감정'])

In [ ]:
train_text_input = Text_Input(train_text)
valid_text_input = Text_Input(valid_text)
test_text_input = Text_Input(test_csv)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
epochs = 5
batch_size = 16
optimizer = AdamW(model.parameters(), lr=5e-6)

In [ ]:
train_loader = DataLoader(train_text_input, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_text_input, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_text_input, batch_size=batch_size, shuffle=True)

In [ ]:
losses = []
accuracies = []

for i in range(epochs):
    total_loss = 0.0
    correct = 0
    total = 0
    batches = 0

    model.train()


for input_ids_batch, attention_masks_batch, y_batch in tqdm(train_loader):
    optimizer.zero_grad()
    y_batch = y_batch.to(device)
    input_ids_batch = input_ids_batch.to(device)
    attention_masks_batch = attention_masks_batch.to(device)
    y_pred = model(input_ids_batch, attention_masks_batch)
    try:
        loss = F.cross_entropy(y_pred, y_batch)
        loss.backward()
        optimizer.step()
    except Exception as e:
        print(e)
        print(input_ids_batch.size(), attention_masks_batch.size(), y_pred.size(), y_batch.size())
        break


    total_loss += loss.item()

    _, predicted = torch.max(y_pred, 1)
    correct += (predicted == y_batch).sum()
    total += len(y_batch)

    batches += 1
    if batches % 100 == 0 and total != 0:
        print("Batch Loss:", total_loss, "Accuracy:", correct / total)
    else:
        accuracies.append(0)

losses.append(total_loss)
accuracies.append(correct / total)
print("Train Loss:", total_loss, "Accuracy:", correct / total)

In [ ]:
losses, accuracies

In [ ]:
# 모델 저장하기
torch.save(model.state_dict(), "./model/1st_koElectraModel.pt")

In [ ]:
model.eval()

test_correct = 0
test_total = 0

for input_ids_batch, attention_masks_batch, y_batch in tqdm(test_loader):
    y_batch = y_batch.to(device)
    y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
    _, predicted = torch.max(y_pred, 1)
    test_correct += (predicted == y_batch).sum()
    test_total += len(y_batch)

print("Accuracy:", test_correct.float() / test_total)

### 문제:

- Colab에서 돌려본 텍스트 모델 KoELECTRA 성능이 생각 외로 형편없다....(최종 점검 결과 약 20%)
- 한 번 더 데이터 확인해 보고 문제 없으면 다른 방법 강구해 보자
- 그리고 앞으로는 train_test_split이나 KFold로 분리한 train data의 일부는 valid, validation으로 지칭하고 원래 분리해 놓은 세트는 test로 지칭. 용어 혼선이 있었음.

# 텍스트 전처리
- 나중에 10차 시도에 잘라 붙일 부분